# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

import collections
# Due to the not working widgets library at Python 3.10+
if not hasattr(collections, 'Mapping'):
    import collections.abc
    collections.Mapping = collections.abc.Mapping
    collections.Sequence = collections.abc.Sequence
    collections.Iterable = collections.abc.Iterable
    collections.Callable = collections.abc.Callable

from azureml.widgets import RunDetails

## Dataset

### Overview


The dataset used in this project is the Heart Failure Clinical Records Dataset, obtained from Kaggle. The file "heart_failure_clinical_records_dataset.csv" contains clinical records of 299 patients who experienced heart failure.

The dataset is firstly downloaded from https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data, then uploaded locally to Azure ML Workspace, and then used directly in the experiments.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'AutoML-HFP-Exp'
project_folder = './automl'

experiment=Experiment(ws, experiment_name)
experiment

quick-starts-ws-294022
aml-quickstarts-294022
westeurope
6971f5ac-8af1-446e-8034-05acea24681f


Name,Workspace,Report Page,Docs Page
AutoML-HFP-Exp,quick-starts-ws-294022,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "heart_failure_clinical_records"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/onuroezcelik/nd00333_Capstone_Project_Machine_Learning_Engineer/blob/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

print("The registered dataset is found: ",found)
df = dataset.to_pandas_dataframe()
df.describe()

The registered dataset is found:  True
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [4]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automlcluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating a new compute cluster.
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [6]:
# automl settings
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# automl config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-HFP-Exp,AutoML_21332f17-dd47-44c5-9f4b-294511134ac6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

Matplotlib is building the font cache; this may take a moment.
2026-01-19 08:44:11.189741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768812252.902592    6069 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768812253.365578    6069 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768812257.701914    6069 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768812257.701957    6069 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768812257.7

A Jupyter Widget

Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-HFP-Exp,AutoML_21332f17-dd47-44c5-9f4b-294511134ac6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_21332f17-dd47-44c5-9f4b-294511134ac6',
 'target': 'automlcluster',
 'status': 'Completed',
 'startTimeUtc': '2026-01-19T08:43:28.679611Z',
 'endTimeUtc': '2026-01-19T09:04:03.443472Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'automlcluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-HFP-Exp","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-294022","workspace_name":"quick-starts-ws-294022","region":"westeurope","compute_target":"automlcluster"

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
results = []

for child in remote_run.get_children():
    name = child.properties.get('run_algorithm')
    metrics = child.get_metrics()
    acc = metrics.get('accuracy', None)

    if acc is not None:
        results.append((name, acc))

results_sorted = sorted(results, key=lambda x: x[1], reverse=True)

for name, acc in results_sorted:
    print(f"{name}: {acc}")

VotingEnsemble: 0.8795402298850575
StackEnsemble: 0.8627586206896553
XGBoostClassifier: 0.859425287356322
XGBoostClassifier: 0.8526436781609196
LightGBM: 0.8526436781609196
RandomForest: 0.8462068965517242
LightGBM: 0.8460919540229886
LightGBM: 0.8393103448275863
LightGBM: 0.8360919540229885
XGBoostClassifier: 0.835977011494253
RandomForest: 0.8359770114942527
LightGBM: 0.8359770114942527
XGBoostClassifier: 0.8358620689655174
XGBoostClassifier: 0.8329885057471265
XGBoostClassifier: 0.8328735632183909
ExtremeRandomTrees: 0.8291954022988506
RandomForest: 0.8290804597701149
RandomForest: 0.8260919540229885
XGBoostClassifier: 0.825977011494253
GradientBoosting: 0.8258620689655173
LogisticRegression: 0.8258620689655173
RandomForest: 0.8227586206896552
LightGBM: 0.8193103448275864
LogisticRegression: 0.8159770114942528
XGBoostClassifier: 0.8126436781609195
XGBoostClassifier: 0.8093103448275862
LogisticRegression: 0.792528735632184
RandomForest: 0.7889655172413793
KNN: 0.7658620689655172
SVM:

## Model Deployment

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
# Save the best model
best_automl_run, best_automl_model= remote_run.get_output()
best_automl_model = best_automl_run.register_model(model_name="best_automl_model", model_path="outputs/")

Package:azureml-automl-runtime, training version:1.61.0, current version:1.60.0
Package:azureml-core, training version:1.61.0.post1, current version:1.60.0
Package:azureml-dataprep, training version:5.4.2, current version:5.1.6
Package:azureml-dataprep-native, training version:42.1.0, current version:41.0.0
Package:azureml-dataprep-rslex, training version:2.25.2, current version:2.22.5
Package:azureml-dataset-runtime, training version:1.61.0, current version:1.60.0
Package:azureml-defaults, training version:1.61.0, current version:1.60.0
Package:azureml-inference-server-http, training version:1.5.0, current version:1.4.0
Package:azureml-interpret, training version:1.61.0, current version:1.60.0
Package:azureml-mlflow, training version:1.61.0.post1, current version:1.60.0
Package:azureml-pipeline-core, training version:1.61.0, current version:1.60.0
Package:azureml-responsibleai, training version:1.61.0, current version:1.60.0
Package:azureml-telemetry, training version:1.61.0, current 

In [11]:
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In the cell below, send a request to the web service you deployed to test it.

In [12]:
environment = best_automl_run.get_environment()
entry_script='scoring.py'

In [14]:
inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "bestmodeldeployv4", [best_automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2026-01-19 11:27:08+00:00 Creating Container Registry if not exists.
2026-01-19 11:27:08+00:00 Use the existing image.
2026-01-19 11:27:13+00:00 Submitting deployment to compute..
2026-01-19 11:27:20+00:00 Checking the status of deployment bestmodeldeployv4..
2026-01-19 11:29:42+00:00 Checking the status of inference endpoint bestmodeldeployv4.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
print(service.get_logs())

2026-01-19T11:29:36,950315535+00:00 - rsyslog/run 
2026-01-19T11:29:36,955224100+00:00 - gunicorn/run 
2026-01-19T11:29:36,956206351+00:00 | gunicorn/run | 
2026-01-19T11:29:36,957193534+00:00 | gunicorn/run | ###############################################
2026-01-19T11:29:36,958177761+00:00 | gunicorn/run | AzureML Container Runtime Information
2026-01-19T11:29:36,967754706+00:00 | gunicorn/run | ###############################################
2026-01-19T11:29:36,970512594+00:00 | gunicorn/run | 
2026-01-19T11:29:36,977016934+00:00 - nginx/run 
2026-01-19T11:29:36,978598607+00:00 | gunicorn/run | 
2026-01-19T11:29:36,979544771+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/usr/local/openmpi/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
2026-01-19T11:29:36,980501747+00:00 | gunicorn/run | PYTHONPATH environment variable: 
2026-01-19T11:29:36,981464995+00:00 | gunicorn/run | 
/opt/miniconda/lib/python3.10/site-p

Testing the AutoML model with the patient data in endpoint.py

In [18]:
%run endpoint.py

{"result": [1, 1]}


In the cell below, print the logs of the web service and delete the service

In [19]:
service.delete()

Running
2026-01-19 12:21:46+00:00 Check and wait for operation (36f488b7-8968-43dc-9f63-2cadfea88f6b) to finish.
2026-01-19 12:21:51+00:00 Deleting service entity.
Succeeded


In [20]:
compute_target.delete()